<a href="https://colab.research.google.com/github/lawrennd/qig-code/blob/main/examples/effective_hamiltonian_derivation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deriving the Effective Hamiltonian from Antisymmetric GENERIC Flow

### Neil D. Lawrence

### December 2025



## Summary

This notebook provides the symbolic derivation for extracting an
explicit effective Hamiltonian $H_{\text{eff}}(\theta)$ from the
antisymmetric (entropy-conserving) sector of the constrained GENERIC
flow.

Given the antisymmetric Jacobian $A(\theta)$ and Lie structure constants
$f_{abc}$, the Hamiltonian coefficients satisfy $$
A_{ab} \theta_b = \sum_c f_{abc} \eta_c.
$$
This determines $\eta = (\eta_1, \ldots, \eta_n)$ such that: $$
H_{\text{eff}}(\theta) = \sum_c \eta_c(\theta) F_c
$$
generates the reversible dynamics via the von Neumann equation
$\dot{\rho} = -i[H_{\text{eff}}, \rho]$.



## 1. Background: GENERIC Decomposition

### 1.1 The Framework

The GENERIC (General Equation for Non-Equilibrium
Reversible-Irreversible Coupling) framework decomposes dynamics into
reversible and irreversible parts. For a quantum exponential family
parameterized by natural parameters $\theta$, the flow in parameter
space is $$
\dot{\theta} = M(\theta),
$$ where $M$ is the flow Jacobian. This decomposes as $$
M = S + A
$$ where

-   $S$ is **symmetric**: $S_{ab} = S_{ba}$ (dissipative/irreversible)
-   $A$ is **antisymmetric**: $A_{ab} = -A_{ba}$
    (conservative/reversible)

### 1.2 Code Setup



import numpy as np
from qig.exponential_family import QuantumExponentialFamily
from qig.structure_constants import compute_structure_constants
from qig.generic import (
    effective_hamiltonian_coefficients,
    effective_hamiltonian_operator
)


In [ ]:
# Create a simple qubit-pair system with Lie-closed basis
exp_fam = QuantumExponentialFamily(n_pairs=1, d=2, pair_basis=True)

print(f"System dimension: {exp_fam.D}")
print(f"Number of parameters: {exp_fam.n_params}")
print(f"Basis operators: {len(exp_fam.operators)}")


The `pair_basis=True` ensures our operators form a **Lie-closed
algebra** (commutators stay within the span), which is essential for the
extraction.



## 2. From Density Matrix to Parameter Flow

### 2.1 The Exponential Family Structure

A quantum exponential family has the form $$
\rho(\theta) = \frac{1}{Z(\theta)} \exp\left(\sum_a \theta_a F_a\right)
$$ where

-   $\theta = (\theta_1, \ldots, \theta_n)$ are natural parameters
-   $F_a$ are Hermitian, traceless basis operators
-   $Z(\theta) = \text{Tr}[\exp(\sum_a \theta_a F_a)]$ is the partition
    function

### 2.2 The Kubo-Mori Derivatives

The fundamental object connecting parameter space to density-matrix
space is the **Kubo-Mori derivative** $$
\frac{\partial \rho}{\partial \theta_a} = \int_0^1 e^{sH(\theta)} \left(F_a - \langle F_a \rangle I\right) e^{(1-s)H(\theta)} \text{d}s
$$ where $H(\theta) = \sum_b \theta_b F_b$ is the Hamiltonian defining
the state.

This is **not** simply the commutator $[F_a, \rho]$, but includes an
operator-ordered integral kernel.

### 2.3 Computing the Kubo-Mori Derivatives


In [ ]:
# Choose a parameter point
np.random.seed(24)
theta = np.random.rand(exp_fam.n_params)

# Get the density matrix
rho = exp_fam.rho_from_theta(theta)
print(f"Density matrix shape: {rho.shape}")
print(f"Tr(ρ) = {np.trace(rho):.6f}")
print(f"Is Hermitian: {np.allclose(rho, rho.conj().T)}")

# Compute Kubo-Mori derivative for first parameter
drho_dtheta_0 = exp_fam.rho_derivative(theta, 0, method='duhamel_spectral')
print(f"\n∂ρ/∂θ_0 shape: {drho_dtheta_0.shape}")
print(f"Is Hermitian: {np.allclose(drho_dtheta_0, drho_dtheta_0.conj().T)}")
print(f"Tr(∂ρ/∂θ_0) = {np.trace(drho_dtheta_0):.2e}")  # Should be ~0



### 2.4 The Fisher Information and Flow Jacobian

The **quantum Fisher information** (QFI) tensor is $$
G_{ab}(\theta) = \text{Tr}\left[\frac{\partial \rho}{\partial \theta_a} \frac{\partial \rho}{\partial \theta_b}\right]_{\text{symmetric part}}.
$$ The full flow Jacobian $M$ encodes how parameters evolve under
dynamics. It can be computed from the Kubo-Mori derivatives.



## 3. The Antisymmetric Part: Reversible Dynamics

### 3.1 Why Antisymmetric Means Reversible

The antisymmetric part $A$ of the flow Jacobian corresponds to
entropy-conserving dynamics. In physics, this means $$
\frac{\text{d}S}{\text{d}t}\bigg|_{\text{reversible}} = 0
$$ For quantum systems, entropy-conserving dynamics are **unitary**
(Hamiltonian) evolution $$
\dot{\rho} = -i[H, \rho].
$$ This is the von Neumann equation. Our goal is to find the explicit
$H_{\text{eff}}(\theta)$ that generates this evolution.

### 3.2 Computing the Antisymmetric Jacobian


In [ ]:
# Compute the antisymmetric part of the flow Jacobian
A = exp_fam.antisymmetric_part(theta, method='duhamel_spectral')

print(f"Antisymmetric Jacobian shape: {A.shape}")
print(f"Max symmetry error: {np.max(np.abs(A + A.T)):.2e}")


In [ ]:

# Visualize the structure
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=big_figsize)

# A matrix
im1 = ax1.imshow(A, cmap='RdBu', vmin=-np.max(np.abs(A)), vmax=np.max(np.abs(A)))
ax.set_title('Antisymmetric Jacobian A')
ax.set_xlabel('Parameter index b')
ax.set_ylabel('Parameter index a')
plt.colorbar(im1, ax=ax)
plt.savefig('./diagrams/antisymmetric_jacobian_A.png', dpi=150)


In [ ]:

fig, ax = plt.subplots(figsize=big_figsize)

# Antisymmetry check
antisym_check = A + A.T
im2 = ax2.imshow(antisym_check, cmap='viridis')
ax.set_title('Antisymmetry Check: A + Aᵀ (should be ~0)')
ax.set_xlabel('Parameter index b')
ax.set_ylabel('Parameter index a')
plt.colorbar(im2, ax=ax)

plt.tight_layout()
plt.savefig('./diagrams/antisymmetric_jacobian_A-AT.png', dpi=150)




## 4. Lie Structure Constants: The Key to Extraction

### 4.1 What Are Structure Constants?

Since our operators form a **Lie algebra**, their commutators close $$
[F_a, F_b] = 2i \sum_c f_{abc} F_c.
$$ The coefficients $f_{abc}$ are the **Lie structure constants**. They
satisfy: - $f_{abc} = -f_{bac}$ (antisymmetry in first two indices) -
Jacobi identity: $\sum_{\text{cyclic}} f_{ab}^d f_{dc}^e = 0$

### 4.2 Computing Structure Constants


In [ ]:
# Compute structure constants for our basis
f_abc = compute_structure_constants(exp_fam.operators)

print(f"Structure constants shape: {f_abc.shape}")
print(f"Antisymmetry check (should be ~0): {np.max(np.abs(f_abc + f_abc.swapaxes(0,1))):.2e}")

# How many non-zero entries?
n_nonzero = np.sum(np.abs(f_abc) > 1e-10)
n_total = f_abc.size
print(f"Non-zero entries: {n_nonzero} / {n_total} ({100*n_nonzero/n_total:.1f}%)")



### 4.3 The Extraction Formula: Geometric Intuition

Here’s the key insight. The antisymmetric flow in parameter space is: $$
\dot{\theta}_a = A_{ab} \theta_b.
$$ This must be equivalent to the von Neumann evolution in
density-matrix space $$
\dot{\rho} = -i[H_{\text{eff}}, \rho].
$$ If $H_{\text{eff}} = \sum_c \eta_c F_c$, then by the chain rule $$
\dot{\rho} = \sum_a \frac{\partial \rho}{\partial \theta_a} \dot{\theta}_a = \sum_a \frac{\partial \rho}{\partial \theta_a} (A_{ab} \theta_b).
$$ The commutator $[F_c, \rho]$ can be related to the parameter-space
flow through the structure constants and Kubo-Mori derivatives. After
careful calculation (involving the Kubo-Mori kernel structure), this
yields $$
A_{ab} \theta_b = \sum_c f_{abc} \eta_c.
$$ This is a **linear system** for the Hamiltonian coefficients
$\eta_c$!



## 5. Solving for the Hamiltonian Coefficients

### 5.1 The Linear System

The extraction formula $A_{ab} \theta_b = \sum_c f_{abc} \eta_c$ can be
rewritten as $$
\text{lhs}_a = \text{rhs}_{ac} \cdot \eta_c
$$ where

-   $\text{lhs}_a = A_{ab} \theta_b$ (vector of length $n$)
-   $\text{rhs}_{ac} = f_{abc}$ (matrix of shape $n \times n$)

This is typically an **overdetermined** system (more equations than
unknowns for Lie algebras). We solve it using least squares.

### 5.2 Code Implementation


In [ ]:
# Extract Hamiltonian coefficients
eta, diagnostics = effective_hamiltonian_coefficients(A, theta, f_abc)

print(f"Hamiltonian coefficients η shape: {eta.shape}")
print(f"Solution residual: {diagnostics['residual']:.2e}")
print(f"Condition number: {diagnostics['condition_number']:.2e}")

# Verify the extraction formula
lhs = A @ theta  # Left-hand side
rhs = np.einsum('abc,c->a', f_abc, eta)  # Right-hand side

extraction_error = np.linalg.norm(lhs - rhs)
print(f"\nExtraction formula error: {extraction_error:.2e}")
print(f"  ||A @ θ||: {np.linalg.norm(lhs):.4e}")
print(f"  ||f @ η||: {np.linalg.norm(rhs):.4e}")



The extraction is *highly accurate* (residual \~1e-7).



## 6. Building the Effective Hamiltonian Operator

### 6.1 From Coefficients to Operator

Given coefficients $\eta_c$, we construct $$
H_{\text{eff}} = \sum_c \eta_c F_c.
$$

This must be

-   **Hermitian**: $H_{\text{eff}}^\dagger = H_{\text{eff}}$ (physical
    observables are Hermitian)
-   **Traceless**: $\text{Tr}[H_{\text{eff}}] = 0$ (gauge freedom in
    Hamiltonian)

### 6.2 Code Implementation


In [ ]:
# Build the effective Hamiltonian operator
H_eff = effective_hamiltonian_operator(eta, exp_fam.operators)

print(f"H_eff shape: {H_eff.shape}")
print(f"Is Hermitian: {np.allclose(H_eff, H_eff.conj().T)}")
print(f"Max Hermiticity error: {np.max(np.abs(H_eff - H_eff.conj().T)):.2e}")
print(f"Trace: {np.trace(H_eff):.2e}")
print(f"Frobenius norm: {np.linalg.norm(H_eff, 'fro'):.4e}")

In [ ]:
# Visualise
fig, ax = plt.subplots(figsize=big_figsize)

# Real part
im1 = ax.imshow(np.real(H_eff), cmap='RdBu',
                  vmin=-np.max(np.abs(H_eff)), vmax=np.max(np.abs(H_eff)))
ax.set_title('Re(H_eff)')
plt.colorbar(im1, ax=ax)
plt.tight_layout()
plt.savefig('./diagrams/effective-hamiltonian_real.png', dpi=150)




In [ ]:
fig, ax = plt.subplots(figsize=big_figsize)
# Imaginary part
im2 = ax.imshow(np.imag(H_eff), cmap='RdBu',
                  vmin=-np.max(np.abs(H_eff)), vmax=np.max(np.abs(H_eff)))
ax.set_title('Im(H_eff)')
plt.colorbar(im2, ax=ax)

plt.tight_layout()
plt.savefig('./diagrams/effective-hamiltonian_imaginary.png', dpi=150)




### 6.3 Physical Interpretation

The eigenvalues of $H_{\text{eff}}$ give the **energy levels** of the
effective system.


In [ ]:
eigenvalues = np.linalg.eigvalsh(H_eff)
print("Energy eigenvalues:")
for i, E in enumerate(eigenvalues):
    print(f"  E_{i} = {E:+.6f}")

# Energy gaps
gaps = np.diff(eigenvalues)
print(f"\nEnergy gaps: {gaps}")
print(f"Ground state: E_0 = {eigenvalues[0]:.6f}")
print(f"Excited state splitting: ΔE = {eigenvalues[-1] - eigenvalues[0]:.6f}")


## 7. Symbolic Form of the Extraction

### 7.1 The Complete Symbolic Expression

Given a parameterized state $\rho(\theta)$ in a quantum exponential
family, the effective Hamiltonian has the explicit symbolic form $$
H_{\text{eff}}(\theta) = \sum_{c=1}^{n} \eta_c(\theta) F_c.
$$ where the coefficients $\eta_c(\theta)$ are determined by solving $$
\sum_c f_{abc} \eta_c = A_{ab}(\theta) \theta_b.
$$ This is a **linear map** from the antisymmetric Jacobian to
Hamiltonian coefficients $$
\eta = \mathcal{L}^{-1}[A \theta],
$$ where $\mathcal{L}$ is the linear operator defined by the structure
constants.

### 7.2 Matrix Form of the Extraction

In matrix notation, define the $n \times n$ matrix $\mathbf{F}$ where $$
\mathbf{F}_{ac} = f_{abc},
$$ (summing over the repeated index $b$ in the Einstein convention).
Then $$
\mathbf{F} \cdot \boldsymbol{\eta} = A \boldsymbol{\theta}.
$$ This is solved via least-squares when overdetermined: $$
\boldsymbol{\eta} = (\mathbf{F}^T \mathbf{F})^{-1} \mathbf{F}^T (A \boldsymbol{\theta}).
$$

### 7.3 Geometric Interpretation

The extraction formula represents a *change of basis* from the natural
parameter tangent space to the Lie algebra.

    Natural parameters θ  →  Parameter flow Aθ  →  Lie algebra element η  →  Hamiltonian H_eff

The structure constants $f_{abc}$ encode the Lie algebra structure and
provide the bridge between parameter-space antisymmetry and
operator-space Hamiltonian structure.



## 8. Symbolic Properties and Identities

### 8.1 Guaranteed Structural Properties

By construction from a Lie algebra, $H_{\text{eff}}$ satisfies:

1.  **Hermiticity**: $H_{\text{eff}}^\dagger = H_{\text{eff}}$ (each
    $F_c$ is Hermitian, $\eta_c \in \mathbb{R}$)
2.  **Tracelessness**: $\text{Tr}[H_{\text{eff}}] = 0$ (each $F_c$ is
    traceless)
3.  **Lie closure**: $[H_{\text{eff}}, F_a] = 2i \sum_b C_{ab} F_b$ for
    some real coefficients $C_{ab}$

These hold **exactly** at the symbolic level.

### 8.2 The Extraction Identity

The fundamental identity relating antisymmetric flow to Hamiltonian
structure: $$
\boxed{A_{ab} \theta_b = \sum_c f_{abc} \eta_c}.
$$ This can be understood as: the antisymmetric Jacobian acting on
parameters produces a vector in tangent space that, when contracted with
structure constants, yields Lie algebra coefficients.

### 8.3 Dependence on Parameter Point

The Hamiltonian coefficients $\eta_c$ are **functions of $\theta$** $$
\eta_c = \eta_c(\theta)
$$ because both $A(\theta)$ and the extraction formula depend on the
parameter point. The full symbolic solution is $$
\eta_c(\theta) = \sum_{a,b} [\mathbf{F}^\dagger (\mathbf{F} \mathbf{F}^\dagger)^{-1}]_{ca} A_{ab}(\theta) \theta_b
$$ where $\mathbf{F}_{ac} = f_{abc}$ is the structure constant matrix.



## 9. Summary: The Symbolic Extraction Pipeline

### 9.1 Input Data

1.  **Exponential family**: Density matrices
    $\rho(\theta) = Z^{-1} \exp(\sum_a \theta_a F_a)$
2.  **Lie-closed basis**: Operators $\{F_a\}$ with commutation
    $[F_a, F_b] = 2i \sum_c f_{abc} F_c$
3.  **Parameter point**: $\theta \in \mathbb{R}^n$
4.  **Antisymmetric Jacobian**: $A(\theta) \in \mathbb{R}^{n \times n}$
    with $A = -A^T$

### 9.2 Symbolic Output

$$
\boxed{H_{\text{eff}}(\theta) = \sum_{c=1}^{n} \eta_c(\theta) F_c}
$$ where $$
\boxed{\eta_c(\theta) = \sum_a [\mathcal{L}^{-1}]_{ca} (A \theta)_a}
$$ and $\mathcal{L}$ is the linear operator $\mathcal{L}_{ac} = f_{abc}$
(structure constant contraction).

### 9.3 Key Properties

The extracted Hamiltonian satisfies.

1.  **Hermiticity**: $H_{\text{eff}}^\dagger = H_{\text{eff}}$ (exactly)
2.  **Tracelessness**: $\text{Tr}[H_{\text{eff}}] = 0$ (exactly)  
3.  **Generates reversible flow**: The antisymmetric part of GENERIC
    corresponds to $\dot{\rho} = -i[H_{\text{eff}}, \rho]$
4.  **Parameter-dependent**: $H_{\text{eff}}$ varies smoothly with
    $\theta$

### 9.4 Practical Usage

**When to use**: - Extracting explicit Hamiltonians from GENERIC
decomposition - Converting parameter-space antisymmetric flow to
operator form - Analyzing energy spectrum of reversible dynamics at a
parameter point

**Computational complexity**: - Structure constants: $O(n^3 d^2)$ where
$d$ is Hilbert dimension - Antisymmetric Jacobian: $O(n^2 d^2)$ -
Extraction (least squares): $O(n^3)$ - Total: Dominated by structure
constant computation for small $n$



## 10. Complete Working Example

Here’s a complete script demonstrating the full pipeline:

``` python
#!/usr/bin/env python3
"""
Complete example: Extract effective Hamiltonian from GENERIC flow
"""
import numpy as np
from qig.exponential_family import QuantumExponentialFamily
from qig.structure_constants import compute_structure_constants
from qig.generic import (
    effective_hamiltonian_coefficients,
    effective_hamiltonian_operator
)

def main():
    # 1. Create quantum exponential family
    print("=" * 60)
    print("EFFECTIVE HAMILTONIAN EXTRACTION")
    print("=" * 60)
    
    exp_fam = QuantumExponentialFamily(n_pairs=1, d=2, pair_basis=True)
    print(f"\nSystem: {exp_fam.n_pairs} qubit pair(s)")
    print(f"Hilbert dimension: {exp_fam.D}")
    print(f"Parameters: {exp_fam.n_params}")
    
    # 2. Choose parameter point
    np.random.seed(42)
    theta = 0.05 * np.random.rand(exp_fam.n_params)
    rho = exp_fam.rho_from_theta(theta)
    
    print(f"\nDensity matrix:")
    print(f"  Tr(ρ) = {np.trace(rho):.6f}")
    print(f"  Purity = {np.trace(rho @ rho):.6f}")
    
    # 3. Compute antisymmetric Jacobian
    print(f"\nComputing GENERIC decomposition...")
    A = exp_fam.antisymmetric_part(theta, method='duhamel_spectral')
    print(f"  Antisymmetric Jacobian A: {A.shape}")
    print(f"  Max antisymmetry error: {np.max(np.abs(A + A.T)):.2e}")
    
    # 4. Compute structure constants
    print(f"\nComputing Lie structure constants...")
    f_abc = compute_structure_constants(exp_fam.operators)
    print(f"  Structure constants f_abc: {f_abc.shape}")
    n_nonzero = np.sum(np.abs(f_abc) > 1e-10)
    print(f"  Non-zero entries: {n_nonzero} / {f_abc.size}")
    
    # 5. Extract Hamiltonian coefficients
    print(f"\nExtracting Hamiltonian coefficients...")
    eta, diagnostics = effective_hamiltonian_coefficients(A, theta, f_abc)
    print(f"  Coefficients η: {eta.shape}")
    print(f"  Residual: {diagnostics['residual']:.2e}")
    print(f"  Condition number: {diagnostics['condition_number']:.2e}")
    
    # Verify extraction formula
    lhs = A @ theta
    rhs = np.einsum('abc,c->a', f_abc, eta)
    print(f"  Extraction error: {np.linalg.norm(lhs - rhs):.2e}")
    
    # 6. Build effective Hamiltonian operator
    print(f"\nBuilding effective Hamiltonian H_eff...")
    H_eff = effective_hamiltonian_operator(eta, exp_fam.operators)
    print(f"  H_eff: {H_eff.shape}")
    print(f"  Hermiticity error: {np.max(np.abs(H_eff - H_eff.conj().T)):.2e}")
    print(f"  Trace: {np.trace(H_eff):.2e}")
    print(f"  Norm: {np.linalg.norm(H_eff, 'fro'):.4e}")
    
    # 7. Analyze energy spectrum
    print(f"\nEnergy spectrum:")
    eigenvalues = np.linalg.eigvalsh(H_eff)
    for i, E in enumerate(eigenvalues):
        print(f"  E_{i} = {E:+.6f}")
    
    # 8. Show symbolic form
    print(f"\nSymbolic form:")
    print(f"  H_eff(θ) = Σ_c η_c(θ) F_c")
    print(f"  where η solves: A(θ)θ = f·η")
    print(f"\n  Explicitly:")
    print(f"    H_eff = ", end="")
    for c in range(min(3, len(eta))):
        if c > 0:
            print(" + ", end="")
        print(f"{eta[c]:.4f} F_{c}", end="")
    if len(eta) > 3:
        print(f" + ... ({len(eta)-3} more terms)")
    else:
        print()
    
    print("\n" + "=" * 60)
    print("✓ Extraction complete!")
    print("=" * 60)

if __name__ == '__main__':
    main()
```

Save this as `extract_hamiltonian.py` and run it to see the full
derivation in action.

## References

1.  **CIP-0009**: “Explicit Hamiltonian Extraction from Antisymmetric
    GENERIC Flow”
2.  **Backlog Task**: `2025-12-08_bch-duhamel-implementation.md` -
    BCH-Duhamel kernel analysis
3.  **Implementation**: `qig/generic.py` - Core extraction functions
4.  **Tests**: `tests/test_generic_hamiltonian.py` - Validation suite
5.  **Theory**: “The Inaccessible Game Origin” - Mathematical
    foundations